In [1]:
import pandas as pd

In [9]:
import nltk

In [14]:
from nltk.tokenize import word_tokenize

In [64]:
import re

In [2]:
train_data = pd.read_csv("./nlp-getting-started/train.csv")

In [3]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


# Number of records
Total records: 7613  
Null keyword: 61  
Null location: 2533

In [4]:
train_data.shape

(7613, 5)

In [5]:
train_data[train_data['keyword'].isnull()].shape

(61, 5)

In [8]:
train_data[train_data['location'].isnull()].shape

(2533, 5)

## Duplicated data
61 tweets are duplicated  

In [24]:
train_data[train_data[['keyword','location','text']].duplicated()]

,id,keyword,location,text,target,n_tokens
48,68,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http:/...,0,18
115,165,aftershock,US,320 [IR] ICEMOON [AFTERSHOCK] | http://t.co/vA...,0,31
119,172,aftershock,Switzerland,320 [IR] ICEMOON [AFTERSHOCK] | http://t.co/TH...,0,31
164,238,airplane%20accident,NaN,Experts in France begin examining airplane deb...,1,22
624,898,bioterrorism,NaN,To fight bioterrorism sir.,0,5
...,...,...,...,...,...,...
6377,9113,suicide%20bomb,Nigeria,#Bestnaijamade: 16yr old PKK suicide bomber wh...,1,20
6378,9114,suicide%20bomb,Nigeria,#Bestnaijamade: 16yr old PKK suicide bomber wh...,1,20
6392,9135,suicide%20bomb,Nigeria,#Bestnaijamade: 16yr old PKK suicide bomber wh...,1,20
6449,9225,suicide%20bombing,NaN,'Suicide bombing at [location named]...' #prem...,1,14


remove duplicated data

In [25]:
train_data = train_data[~train_data[['keyword','location','text']].duplicated()]

In [26]:
train_data.shape

(7552, 6)

# Keyword

222 types of keywords exists  
some keywords has % but otherwise, they seems to be proper data.

In [29]:
train_data['keyword'].unique(), len(train_data['keyword'].unique())

(array([nan, 'ablaze', 'accident', 'aftershock', 'airplane%20accident',
        'ambulance', 'annihilated', 'annihilation', 'apocalypse',
        'armageddon', 'army', 'arson', 'arsonist', 'attack', 'attacked',
        'avalanche', 'battle', 'bioterror', 'bioterrorism', 'blaze',
        'blazing', 'bleeding', 'blew%20up', 'blight', 'blizzard', 'blood',
        'bloody', 'blown%20up', 'body%20bag', 'body%20bagging',
        'body%20bags', 'bomb', 'bombed', 'bombing', 'bridge%20collapse',
        'buildings%20burning', 'buildings%20on%20fire', 'burned',
        'burning', 'burning%20buildings', 'bush%20fires', 'casualties',
        'casualty', 'catastrophe', 'catastrophic', 'chemical%20emergency',
        'cliff%20fall', 'collapse', 'collapsed', 'collide', 'collided',
        'collision', 'crash', 'crashed', 'crush', 'crushed', 'curfew',
        'cyclone', 'damage', 'danger', 'dead', 'death', 'deaths', 'debris',
        'deluge', 'deluged', 'demolish', 'demolished', 'demolition',
       

To reduce duplicated data, let's remitize the keywords

In [134]:
from nltk.stem import WordNetLemmatizer

In [135]:
lemmatizer = WordNetLemmatizer()

In [139]:
train_data['keyword'] = train_data['keyword'].fillna('')

C:\Users\jaydenpark\AppData\Local\Temp\2\ipykernel_7076\1428591074.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['keyword'] = train_data['keyword'].fillna('')


In [142]:
train_data['keyword'] = train_data['keyword'].str.replace('%', ' ')

C:\Users\jaydenpark\AppData\Local\Temp\2\ipykernel_7076\1594350304.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['keyword'] = train_data['keyword'].str.replace('%', ' ')


In [145]:
train_data['keyword'] = train_data['keyword'].str.replace('\d', '', regex=True)

C:\Users\jaydenpark\AppData\Local\Temp\2\ipykernel_7076\3737823605.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['keyword'] = train_data['keyword'].str.replace('\d', '', regex=True)


In [146]:
train_data['keyword_lem'] = train_data['keyword'].apply(lambda x: lemmatizer.lemmatize(x))

C:\Users\jaydenpark\AppData\Local\Temp\2\ipykernel_7076\3258151759.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['keyword_lem'] = train_data['keyword'].apply(lambda x: lemmatizer.lemmatize(x))


In [148]:
train_data['keyword_lem'].unique(), len(train_data['keyword_lem'].unique())

(array(['', 'ablaze', 'accident', 'aftershock', 'airplane accident',
        'ambulance', 'annihilated', 'annihilation', 'apocalypse',
        'armageddon', 'army', 'arson', 'arsonist', 'attack', 'attacked',
        'avalanche', 'battle', 'bioterror', 'bioterrorism', 'blaze',
        'blazing', 'bleeding', 'blew up', 'blight', 'blizzard', 'blood',
        'bloody', 'blown up', 'body bag', 'body bagging', 'body bags',
        'bomb', 'bombed', 'bombing', 'bridge collapse',
        'buildings burning', 'buildings on fire', 'burned', 'burning',
        'burning buildings', 'bush fires', 'casualty', 'catastrophe',
        'catastrophic', 'chemical emergency', 'cliff fall', 'collapse',
        'collapsed', 'collide', 'collided', 'collision', 'crash',
        'crashed', 'crush', 'crushed', 'curfew', 'cyclone', 'damage',
        'danger', 'dead', 'death', 'debris', 'deluge', 'deluged',
        'demolish', 'demolished', 'demolition', 'derail', 'derailed',
        'derailment', 'desolate', 'des

# Location

3342 locations exists

In [30]:
train_data['location'].unique(), len(train_data['location'].unique())

(array([nan, 'Birmingham', 'Est. September 2012 - Bristol', ...,
        'Vancouver, Canada', 'London ', 'Lincoln'], dtype=object),
 3342)

In [63]:
train_data[['location']].sample(10)

,location
5066,America of Founding Fathers
742,#expelcl*y
4176,NaN
2070,NaN
5866,Florida Forever
4184,Arizona
2292,London
1253,CORNFIELDS
6757,Toronto
1784,NaN


In [65]:
remove_special = re.compile("[.;:!\'?,\"()\d\[\]<>%#\^&=@\+\-\$\#\\/*]")

In [74]:
import numpy as np

In [83]:
import math

In [97]:
def loc_prep(data):
    data = data.lower()
    data = remove_special.sub(' ', data)
    data = re.sub(' +', ' ', data)
    return data

In [94]:
train_data.loc[train_data['location'].isnull() ,'location_prep'] = ''

In [98]:
train_data['location_prep'] = train_data['location_prep'].apply(loc_prep)

C:\Users\jaydenpark\AppData\Local\Temp\2\ipykernel_7076\1773747410.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['location_prep'] = train_data['location_prep'].apply(loc_prep)


In [99]:
train_data['location_prep'].unique(), len(train_data['location_prep'].unique())

(array(['', 'birmingham', 'est september bristol', ..., 'denton texas',
        ' newcastleupontyne uk', 'lincoln'], dtype=object),
 3076)

location data is not accurate.  
some location data such as jupiter, everywhere seems no meaning.  
Also there are many records that has no location.  
It would be better to focus on keywords and text  

In [132]:
train_data[['location_prep']].sample(10)

,location_prep
6711,jupiter
529,new york ny
6922,nyc
2092,
914,under the blanket
5813,
470,halton ontario
3392,
6149,diamondville
1859,everywhere


## Word tokens of text

In [17]:
train_data['n_tokens'] = train_data['text'].apply(lambda x: len(word_tokenize(x)))

In [19]:
train_data['n_tokens'].describe()

count    7613.000000
mean       18.914226
std         6.871321
min         1.000000
25%        14.000000
50%        19.000000
75%        24.000000
max        72.000000
Name: n_tokens, dtype: float64

In [ ]:
train_data['']